In [2]:
import pandas as pd
df_test=pd.read_csv('/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_test.csv',encoding="latin-1")
df_train=pd.read_csv('/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_train.csv',encoding="latin-1")
df_test=df_test.drop(['UserName','ScreenName','Location'],axis=1)
df_train=df_train.drop(['UserName','ScreenName','Location'],axis=1)
df_test.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [4]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

df_test['Sentiment'] = le.fit_transform(df_test['Sentiment'])
df_train['Sentiment'] = le.fit_transform(df_train['Sentiment'])


In [5]:
#start splitting:
from sklearn.model_selection import train_test_split 
from transformers import AutoTokenizer, AutoModelForSequenceClassification

train_df, val_df = train_test_split(df_train, test_size=0.2, random_state=42)
tokenizer = AutoTokenizer.from_pretrained('prajjwal1/bert-tiny')
num_label = df_train['Sentiment'].nunique()
model = AutoModelForSequenceClassification.from_pretrained('prajjwal1/bert-tiny', num_labels=num_label)

train_token = tokenizer(train_df['OriginalTweet'].tolist(), truncation=True, padding='max_length', max_length=128)
val_token = tokenizer(val_df['OriginalTweet'].tolist(), truncation=True, padding='max_length', max_length=128)
test_token = tokenizer(df_test['OriginalTweet'].tolist(), truncation=True, padding='max_length', max_length=128)

train_labels = train_df['Sentiment'].tolist()
val_labels = val_df['Sentiment'].tolist()
test_labels = df_test['Sentiment'].tolist() 


2025-08-13 18:28:36.922206: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755109716.950515     222 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755109716.959178     222 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
import torch 
from torch.utils.data import Dataset,DataLoader

class SimpleDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
train_dataset = SimpleDataset(train_token, train_labels)
val_dataset   = SimpleDataset(val_token, val_labels)
test_dataset  = SimpleDataset(test_token, test_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=16)
test_loader  = DataLoader(test_dataset, batch_size=16)

In [8]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [9]:
from tqdm import tqdm

num_epochs = 3  
model.train()  

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        batch = {k: v.to(device) for k, v in batch.items()}
        
        outputs = model(**batch)
        loss = outputs.loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loop.set_description(f"Loss: {loss.item():.4f}")



Epoch 1/3


Loss: 1.2298: 100%|██████████| 2058/2058 [00:26<00:00, 78.32it/s]



Epoch 2/3


Loss: 0.7254: 100%|██████████| 2058/2058 [00:24<00:00, 84.72it/s]



Epoch 3/3


Loss: 1.2935: 100%|██████████| 2058/2058 [00:23<00:00, 86.52it/s]


In [11]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        preds = torch.argmax(outputs.logits, dim=-1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch['labels'].cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-score: {f1:.4f}")
submission = pd.DataFrame({
    'OriginalTweet': df_test['OriginalTweet'],
    'Sentiment_True': df_test['Sentiment'],
    'Sentiment_Pred': all_preds
})

submission.to_csv('test_predictions.csv', index=False)
print("CSV file 'test_predictions.csv' saved successfully!")


Test Accuracy: 0.6685
Test Precision: 0.6771
Test Recall: 0.6685
Test F1-score: 0.6700
CSV file 'test_predictions.csv' saved successfully!
